In [48]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# scaler 
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler

# models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import LinearSVC

# pipeline
from sklearn.pipeline import Pipeline 
from imblearn.pipeline import Pipeline as Pipeline_imb 

# resampling
from imblearn.over_sampling import RandomOverSampler 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import NearMiss
from imblearn.combine import SMOTEENN
from imblearn.combine import SMOTETomek

# model selection
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

# scoring
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# pca
from sklearn.decomposition import PCA

import warnings
warnings.filterwarnings('ignore')

In [63]:
# 데이터 불러오기
socar_df = pd.read_csv("insurance_fraud_detect_data.csv")
pd.set_option('display.max_columns', len(socar_df.columns))
socar_df.head()

,fraud_YN,car_model,sharing_type,age_group,has_previous_accident,cumulative_use_count,b2b,accident_ratio,pf_type,socarpass,socarsave,start_hour,duration,accident_hour,repair_cost,insure_cost,accident_location,car_part1,car_part2,repair_cnt,acc_type1,insurance_site_aid_YN,police_site_aid_YN,total_prsn_cnt,test_set
0,0,2,1,3,0,2,0,100,2,0,0,4,2,4,1.302547e+05,0,1,0,0,1,0,0,0,-1,0
1,0,1,0,4,0,4,0,0,3,1,0,1,4,4,0.000000e+00,0,1,1,1,2,1,1,1,0,1
2,0,1,0,4,1,4,0,100,3,1,0,1,4,6,2.581306e+05,12925040,3,1,0,2,1,1,1,0,1
3,0,3,1,3,0,4,0,0,2,0,0,3,3,5,1.341009e+06,0,3,1,0,1,0,0,0,-1,0
4,0,1,0,1,0,1,0,100,1,0,0,1,1,1,5.240306e+05,1209410,1,0,0,2,0,0,0,-1,0


In [65]:
# 컬럼 삭제하기 
socar_copy1 = socar_df.copy()
socar_copy1.drop(['sharing_type', 'b2b', 'pf_type', 'socarsave',\
                  'car_part1', 'car_part2', 'accident_location', 'repair_cnt'], axis=1, inplace=True)

In [66]:
# accident_ratio 분류하기
for i, j in socar_copy1.iterrows():
    if j['accident_ratio'] < 40:
        socar_copy1.loc[i, 'accident_ratio'] = 0
    else:
        socar_copy1.loc[i, 'accident_ratio'] = 1

socar_copy1['accident_ratio'].value_counts()

1    14345
0     1655
Name: accident_ratio, dtype: int64

In [67]:
# repair_cost, insure_cost 로그 스케일링
insure_log = np.log1p(socar_copy1["insure_cost"])
repair_log = np.log1p(socar_copy1["repair_cost"])

socar_copy1["insure_cost"] = insure_log
socar_copy1["repair_cost"] = repair_log
socar_copy1.head()

,fraud_YN,car_model,age_group,has_previous_accident,cumulative_use_count,accident_ratio,socarpass,start_hour,duration,accident_hour,repair_cost,insure_cost,acc_type1,insurance_site_aid_YN,police_site_aid_YN,total_prsn_cnt,test_set
0,0,2,3,0,2,1,0,4,2,4,11.777255,0.000000,0,0,0,-1,0
1,0,1,4,0,4,0,1,1,4,4,0.000000,0.000000,1,1,1,0,1
2,0,1,4,1,4,1,1,1,4,6,12.461225,16.374677,1,1,1,0,1
3,0,3,3,0,4,0,0,3,3,5,14.108934,0.000000,0,0,0,-1,0
4,0,1,1,0,1,1,0,1,1,1,13.169307,14.005644,0,0,0,-1,0


In [68]:
# 원핫인코딩
socar_copy1 = pd.get_dummies(socar_copy1, columns=["car_model", "insurance_site_aid_YN",\
                                                  "police_site_aid_YN", "total_prsn_cnt"])

In [69]:
# train_set, test_set 분리
train_set = socar_copy1[socar_copy1["test_set"] == 0]
test_set = socar_copy1[socar_copy1["test_set"] == 1]

# "test_set" 컬럼 삭제
train_set = train_set.drop("test_set", axis=1)
test_set = test_set.drop("test_set", axis=1)

In [70]:
# features와 target분리
X_features = train_set.drop("fraud_YN", axis=1)
y_target = train_set["fraud_YN"]
X_test = test_set.drop("fraud_YN", axis=1)
y_test = test_set["fraud_YN"]

In [71]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_features, y_target,\
                                                      test_size=0.2, random_state=13, stratify=y_target)

In [72]:
# 샘플링 RandomOverSampler

over_sampling = RandomOverSampler(random_state=13)
X_train_over, y_train_over = over_sampling.fit_sample(X_train, y_train)

In [73]:
# pipeline

lr_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ('pca', PCA(n_components=5)),
    ("clf", LogisticRegression(random_state=13))])

dt_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=5)),
    ("clf", DecisionTreeClassifier(random_state=13))])

rf_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ('pca', PCA(n_components=5)),
    ("clf", RandomForestClassifier(random_state=13))])

lgbm_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=5)),
    ("clf", LGBMClassifier(random_state=13))])

svm_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ('pca', PCA(n_components=5)),
    ("clf", LinearSVC(random_state=13))])



# grid search params
lr_param_grid = [{'clf__C': [0.1, 1, 10]}]

dt_param_grid = [{'clf__criterion':['gini', 'entropy'], 'clf__max_depth':[None,2,3,4,5,6]}]

rf_param_grid = [{'clf__n_estimators': [100, 200, 400, 800]}]

lgbm_param_grid = [{'clf__n_estimators': [50, 100, 200, 400], 'clf__num_leaves': [4, 8, 16, 32]}]

svm_param_grid = [{"clf__C": [0.1, 1, 10]}]



kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=13)

# grid search CV
LR = GridSearchCV(lr_pipe, lr_param_grid, cv=kfold, scoring = 'recall')
DT = GridSearchCV(dt_pipe, dt_param_grid, cv=kfold, scoring = 'recall')
RF = GridSearchCV(rf_pipe, rf_param_grid, cv=kfold, scoring = 'recall')
LGBM = GridSearchCV(lgbm_pipe, lgbm_param_grid, cv=kfold, scoring = 'recall')
SVM = GridSearchCV(svm_pipe, svm_param_grid, cv=kfold, scoring = 'recall')

In [74]:
def get_score(X_train_over, y_train_over):    
    grids = []
    grids.append(('LogisticRe', LR))
    grids.append(('DecisionTree', DT))
    grids.append(('RandomForest', RF))
    grids.append(('LightGBM', LGBM))
    grids.append(('SVC', SVM))
    
                  
    valid_accuracy = []
    valid_recall = []
    valid_precision = []
    valid_f1 = []
    valid_roc = []
    test_accuracy = []
    test_recall = []
    test_precision = []
    test_f1 = []
    test_roc = []
    names = []
    
    for name, grid in grids:
        clf = grid
        clf.fit(X_train_over, y_train_over)
        
        y_pred_valid = clf.predict(X_valid)
        y_pred_test = clf.predict(X_test)
    
        names.append(name)
        valid_accuracy.append(accuracy_score(y_valid, y_pred_valid))
        valid_recall.append(recall_score(y_valid, y_pred_valid))
        valid_precision.append(precision_score(y_valid, y_pred_valid))
        valid_f1.append(f1_score(y_valid, y_pred_valid))
        valid_roc.append(roc_auc_score(y_valid, y_pred_valid))
        
        test_accuracy.append(accuracy_score(y_test, y_pred_test))
        test_recall.append(recall_score(y_test, y_pred_test))
        test_precision.append(precision_score(y_test, y_pred_test))
        test_f1.append(f1_score(y_test, y_pred_test))
        test_roc.append(roc_auc_score(y_test, y_pred_test))
    
    result = pd.DataFrame({'model name':names, 
                           'valid accuracy':valid_accuracy,
                           'valid precision':valid_precision,
                           'valid recall':valid_recall,
                           'valid f1':valid_f1,
                           'valid roc':valid_roc,
                           'test accuracy':test_accuracy,
                           'test precision':test_precision,
                           'test recall':test_recall,
                           'test f1':test_f1,
                           'test roc':test_roc}) 
    return result

In [75]:
get_score(X_train_over, y_train_over)

,model name,valid accuracy,valid precision,valid recall,valid f1,valid roc,test accuracy,test precision,test recall,test f1,test roc
0,LogisticRe,0.669255,0.003525,0.428571,0.006993,0.549241,0.579942,0.003049,0.571429,0.006065,0.575695
1,DecisionTree,0.634317,0.001067,0.142857,0.002119,0.389257,0.619673,0.003367,0.571429,0.006695,0.595605
2,RandomForest,0.997283,0.000000,0.000000,0.000000,0.500000,0.997437,0.000000,0.000000,0.000000,0.499839
3,LightGBM,0.926242,0.005405,0.142857,0.010417,0.535617,0.929189,0.004630,0.142857,0.008969,0.536907
4,SVC,0.684394,0.003695,0.428571,0.007326,0.556831,0.568728,0.002970,0.571429,0.005908,0.570075


In [76]:
# SMOTE sampling
over_sampling = SMOTE(random_state=13)
X_train_over, y_train_over = over_sampling.fit_sample(X_train, y_train)

In [77]:
get_score(X_train_over, y_train_over)

,model name,valid accuracy,valid precision,valid recall,valid f1,valid roc,test accuracy,test precision,test recall,test f1,test roc
0,LogisticRe,0.846273,0.002558,0.142857,0.005025,0.495524,0.925665,0.004405,0.142857,0.008547,0.535141
1,DecisionTree,0.987578,0.000000,0.000000,0.000000,0.495134,0.987504,0.000000,0.000000,0.000000,0.494862
2,RandomForest,0.993401,0.000000,0.000000,0.000000,0.498054,0.995835,0.000000,0.000000,0.000000,0.499037
3,LightGBM,0.987966,0.000000,0.000000,0.000000,0.495329,0.990388,0.000000,0.000000,0.000000,0.496307
4,SVC,0.853649,0.002688,0.142857,0.005277,0.499221,0.937840,0.000000,0.000000,0.000000,0.469974


In [80]:
# RandomUnderSampling

from imblearn.under_sampling import RandomUnderSampler 

under_sampling = RandomUnderSampler(random_state=13)
X_train_under, y_train_under = under_sampling.fit_sample(X_train, y_train)

In [81]:
get_score(X_train_under, y_train_under)

,model name,valid accuracy,valid precision,valid recall,valid f1,valid roc,test accuracy,test precision,test recall,test f1,test roc
0,LogisticRe,0.621506,0.004098,0.571429,0.008138,0.596536,0.438321,0.002281,0.571429,0.004543,0.504725
1,DecisionTree,0.547360,0.005124,0.857143,0.010187,0.701830,0.468760,0.001208,0.285714,0.002407,0.377443
2,RandomForest,0.610637,0.004970,0.714286,0.009872,0.662320,0.507209,0.001952,0.428571,0.003886,0.467979
3,LightGBM,0.563276,0.004433,0.714286,0.008811,0.638575,0.461391,0.001786,0.428571,0.003557,0.445018
4,SVC,0.635870,0.004260,0.571429,0.008457,0.603737,0.452419,0.001756,0.428571,0.003499,0.440522


In [87]:
from imblearn.combine import SMOTETomek

comb_sampling = SMOTETomek(random_state=13)
X_train_comb, y_train_comb = comb_sampling.fit_sample(X_train, y_train)

In [88]:
get_score(X_train_comb, y_train_comb)

,model name,valid accuracy,valid precision,valid recall,valid f1,valid roc,test accuracy,test precision,test recall,test f1,test roc
0,LogisticRe,0.841615,0.002481,0.142857,0.004878,0.493188,0.923102,0.0,0.0,0.0,0.462588
1,DecisionTree,0.986413,0.000000,0.000000,0.000000,0.494550,0.988145,0.0,0.0,0.0,0.495183
2,RandomForest,0.993789,0.000000,0.000000,0.000000,0.498248,0.994873,0.0,0.0,0.0,0.498555
3,LightGBM,0.960016,0.000000,0.000000,0.000000,0.481316,0.971163,0.0,0.0,0.0,0.486673
4,SVC,0.844332,0.002525,0.142857,0.004963,0.494550,0.923102,0.0,0.0,0.0,0.462588


In [89]:
lr_pipe = Pipeline([
    ("scaler", RobustScaler()),
    ('pca', PCA(n_components=5)),
    ("clf", LogisticRegression(random_state=13))])

dt_pipe = Pipeline([
    ('scaler', RobustScaler()),
    ('pca', PCA(n_components=5)),
    ("clf", DecisionTreeClassifier(random_state=13))])

rf_pipe = Pipeline([
    ("scaler", RobustScaler()),
    ('pca', PCA(n_components=5)),
    ("clf", RandomForestClassifier(random_state=13))])

lgbm_pipe = Pipeline([
    ('scaler', RobustScaler()),
    ('pca', PCA(n_components=5)),
    ("clf", LGBMClassifier(random_state=13))])

svm_pipe = Pipeline([
    ("scaler", RobustScaler()),
    ('pca', PCA(n_components=5)),
    ("clf", LinearSVC(random_state=13))])



# grid search params
lr_param_grid = [{'clf__C': [0.1, 1, 10]}]

dt_param_grid = [{'clf__criterion':['gini', 'entropy'], 'clf__max_depth':[None,2,3,4,5,6]}]

rf_param_grid = [{'clf__n_estimators': [100, 200, 400, 800]}]

lgbm_param_grid = [{'clf__n_estimators': [50, 100, 200, 400], 'clf__num_leaves': [4, 8, 16, 32]}]

svm_param_grid = [{"clf__C": [0.1, 1, 10]}]



kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=13)

# grid search CV
LR = GridSearchCV(lr_pipe, lr_param_grid, cv=kfold, scoring = 'recall')
DT = GridSearchCV(dt_pipe, dt_param_grid, cv=kfold, scoring = 'recall')
RF = GridSearchCV(rf_pipe, rf_param_grid, cv=kfold, scoring = 'recall')
LGBM = GridSearchCV(lgbm_pipe, lgbm_param_grid, cv=kfold, scoring = 'recall')
SVM = GridSearchCV(svm_pipe, svm_param_grid, cv=kfold, scoring = 'recall')

In [90]:
# 샘플링 RandomOverSampler

over_sampling = RandomOverSampler(random_state=13)
X_train_over, y_train_over = over_sampling.fit_sample(X_train, y_train)

In [91]:
get_score(X_train_over, y_train_over)

,model name,valid accuracy,valid precision,valid recall,valid f1,valid roc,test accuracy,test precision,test recall,test f1,test roc
0,LogisticRe,0.659938,0.004561,0.571429,0.009050,0.615804,0.482538,0.001859,0.428571,0.003701,0.455615
1,DecisionTree,0.993012,0.000000,0.000000,0.000000,0.497859,0.995194,0.000000,0.000000,0.000000,0.498715
2,RandomForest,0.997283,0.000000,0.000000,0.000000,0.500000,0.997437,0.000000,0.000000,0.000000,0.499839
3,LightGBM,0.946040,0.007463,0.142857,0.014184,0.545543,0.925344,0.008696,0.285714,0.016878,0.606248
4,SVC,0.663820,0.004614,0.571429,0.009153,0.617750,0.485742,0.001870,0.428571,0.003724,0.457221


In [114]:
# 컬럼 삭제하기 
socar_copy2 = socar_df.copy()
socar_copy2.drop(['sharing_type', 'b2b', 'pf_type', 'socarsave',\
                  'car_part2'], axis=1, inplace=True)

In [115]:
# accident_ratio 분류하기
for i, j in socar_copy2.iterrows():
    if j['accident_ratio'] < 40:
        socar_copy2.loc[i, 'accident_ratio'] = 0
    else:
        socar_copy2.loc[i, 'accident_ratio'] = 1

socar_copy2['accident_ratio'].value_counts()

1    14345
0     1655
Name: accident_ratio, dtype: int64

In [106]:
socar_copy2[socar_copy2['fraud_YN'] == 1]['insure_cost'].mean()

982695.3902439025

In [105]:
socar_copy2["insure_cost"].mean()

859436.726

In [107]:
socar_copy2["repair_cost"].mean()

285011.619115965

In [116]:
cost_0 = socar_copy2[(socar_copy2['insure_cost'] == 0) & (socar_copy2['repair_cost'] == 0) &\
            (socar_copy2['repair_cnt'] > 0)].index

In [117]:
cost_0

Int64Index([    1,     8,    14,    15,    20,    23,    28,    31,    32,
               33,
            ...
            15973, 15976, 15979, 15981, 15983, 15987, 15993, 15995, 15997,
            15998],
           dtype='int64', length=3992)

In [118]:
socar_copy2.loc[cost_0, 'insure_cost'] = socar_copy2["insure_cost"].mean()

In [119]:
socar_copy2.loc[cost_0, 'repair_cost'] = socar_copy2["repair_cost"].mean()

In [123]:
# repair_cost, insure_cost 로그 스케일링
insure_log = np.log1p(socar_copy2["insure_cost"])
repair_log = np.log1p(socar_copy2["repair_cost"])

socar_copy2["insure_cost"] = insure_log
socar_copy2["repair_cost"] = repair_log
socar_copy2.head()

,fraud_YN,car_model,age_group,has_previous_accident,cumulative_use_count,accident_ratio,socarpass,start_hour,duration,accident_hour,repair_cost,insure_cost,accident_location,car_part1,repair_cnt,acc_type1,insurance_site_aid_YN,police_site_aid_YN,total_prsn_cnt,test_set
0,0,2,3,0,2,1,0,4,2,4,11.777255,0.000000,1,0,1,0,0,0,-1,0
1,0,1,4,0,4,0,1,1,4,4,12.560289,13.664034,1,1,2,1,1,1,0,1
2,0,1,4,1,4,1,1,1,4,6,12.461225,16.374677,3,1,2,1,1,1,0,1
3,0,3,3,0,4,0,0,3,3,5,14.108934,0.000000,3,1,1,0,0,0,-1,0
4,0,1,1,0,1,1,0,1,1,1,13.169307,14.005644,1,0,2,0,0,0,-1,0


In [124]:
socar_copy2.columns

Index(['fraud_YN', 'car_model', 'age_group', 'has_previous_accident',
       'cumulative_use_count', 'accident_ratio', 'socarpass', 'start_hour',
       'duration', 'accident_hour', 'repair_cost', 'insure_cost',
       'accident_location', 'car_part1', 'repair_cnt', 'acc_type1',
       'insurance_site_aid_YN', 'police_site_aid_YN', 'total_prsn_cnt',
       'test_set'],
      dtype='object')

In [125]:
# 원핫인코딩
socar_copy2 = pd.get_dummies(socar_copy2, columns=["car_model", "car_part1", "insurance_site_aid_YN",\
                                                  "total_prsn_cnt"])

In [126]:
# train_set, test_set 분리
train_set = socar_copy2[socar_copy1["test_set"] == 0]
test_set = socar_copy2[socar_copy1["test_set"] == 1]

# "test_set" 컬럼 삭제
train_set = train_set.drop("test_set", axis=1)
test_set = test_set.drop("test_set", axis=1)

In [127]:
# features와 target분리
X_features = train_set.drop("fraud_YN", axis=1)
y_target = train_set["fraud_YN"]
X_test = test_set.drop("fraud_YN", axis=1)
y_test = test_set["fraud_YN"]

In [128]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_features, y_target,\
                                                      test_size=0.2, random_state=13, stratify=y_target)

In [129]:
# 샘플링 RandomOverSampler

over_sampling = RandomOverSampler(random_state=13)
X_train_over, y_train_over = over_sampling.fit_sample(X_train, y_train)

In [132]:
# pipeline

lr_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(random_state=13))])

dt_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ("clf", DecisionTreeClassifier(random_state=13))])

rf_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", RandomForestClassifier(random_state=13))])

lgbm_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ("clf", LGBMClassifier(random_state=13))])

svm_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LinearSVC(random_state=13))])



# grid search params
lr_param_grid = [{'clf__C': [0.1, 1, 10]}]

dt_param_grid = [{'clf__criterion':['gini', 'entropy'], 'clf__max_depth':[None,2,3,4,5,6]}]

rf_param_grid = [{'clf__n_estimators': [100, 200, 400, 800]}]

lgbm_param_grid = [{'clf__n_estimators': [50, 100, 200, 400], 'clf__num_leaves': [4, 8, 16, 32]}]

svm_param_grid = [{"clf__C": [0.1, 1, 10]}]



kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=13)

# grid search CV
LR = GridSearchCV(lr_pipe, lr_param_grid, cv=kfold, scoring = 'recall')
DT = GridSearchCV(dt_pipe, dt_param_grid, cv=kfold, scoring = 'recall')
RF = GridSearchCV(rf_pipe, rf_param_grid, cv=kfold, scoring = 'recall')
LGBM = GridSearchCV(lgbm_pipe, lgbm_param_grid, cv=kfold, scoring = 'recall')
SVM = GridSearchCV(svm_pipe, svm_param_grid, cv=kfold, scoring = 'recall')

In [131]:
# ss, pca=5
get_score(X_train_over, y_train_over)

,model name,valid accuracy,valid precision,valid recall,valid f1,valid roc,test accuracy,test precision,test recall,test f1,test roc
0,LogisticRe,0.701475,0.003906,0.428571,0.007742,0.565395,0.626081,0.001718,0.285714,0.003416,0.456280
1,DecisionTree,0.507764,0.001581,0.285714,0.003145,0.397042,0.548542,0.002131,0.428571,0.004240,0.488692
2,RandomForest,0.997283,0.000000,0.000000,0.000000,0.500000,0.997757,0.000000,0.000000,0.000000,0.500000
3,LightGBM,0.970497,0.000000,0.000000,0.000000,0.486571,0.958347,0.000000,0.000000,0.000000,0.480250
4,SVC,0.688665,0.004981,0.571429,0.009877,0.630206,0.640500,0.001787,0.285714,0.003552,0.463506


In [133]:
# ss
get_score(X_train_over, y_train_over)

,model name,valid accuracy,valid precision,valid recall,valid f1,valid roc,test accuracy,test precision,test recall,test f1,test roc
0,LogisticRe,0.695652,0.005096,0.571429,0.010101,0.633710,0.670618,0.001951,0.285714,0.003876,0.478599
1,DecisionTree,0.994565,0.000000,0.000000,0.000000,0.498638,0.995514,0.000000,0.000000,0.000000,0.498876
2,RandomForest,0.997283,0.000000,0.000000,0.000000,0.500000,0.997757,0.000000,0.000000,0.000000,0.500000
3,LightGBM,0.841615,0.002481,0.142857,0.004878,0.493188,0.832426,0.005747,0.428571,0.011342,0.630952
4,SVC,0.694876,0.005083,0.571429,0.010076,0.633320,0.675745,0.001982,0.285714,0.003937,0.481168
